In [1]:
import pandas as pd
import os

from tqdm.notebook import tqdm

import pickle
from pathlib import Path  # only supported in Python 3

In [2]:
src='lt'
trg='en'
lp_src = src + trg
lp = src + '-' + trg

In [3]:
SRC_PREFIX = 'wmt19-metrics-task-package/input/wmt19-submitted-data-v3/txt/sources/'
REF_PREFIX = 'wmt19-metrics-task-package/input/wmt19-submitted-data-v3/txt/references/'
SYS_PREFIX = 'wmt19-metrics-task-package/input/wmt19-submitted-data-v3/txt/system-outputs/newstest2019/'

In [4]:
sources = [line.strip() for line in open(SRC_PREFIX+'newstest2019-{}-src.{}'.format(lp_src, src), encoding="utf-8")]

In [5]:
references = [line.strip() for line in open(REF_PREFIX+'newstest2019-{}-ref.{}'.format(lp_src, trg), encoding="utf-8")]

In [6]:
#extra_references = [line.strip() for line in open('en-de/wmt19-ende-ar.ref.txt', encoding="utf-8")]

### Systems:

In [7]:
sys_outputs_files = os.listdir(SYS_PREFIX+'{}/'.format(lp))
systems = ['.'.join(submission_name.split('.')[1:-1]) for submission_name in sys_outputs_files]

In [8]:
def get_system_outputs(lp):
    sys_outputs_files = os.listdir(SYS_PREFIX+f'{lp}/')
    systems = ['.'.join(submission_name.split('.')[1:-1]) for submission_name in sys_outputs_files]
    system_outputs = {}
    for system in sys_outputs_files:
        system_name = '.'.join(system.split('.')[1:-1])
        system_outputs[system_name] = [
            line.strip() for line in open(SYS_PREFIX+f'{lp}/'+system, encoding="utf-8")
        ]
    return system_outputs

In [9]:
#en_de_data = pd.DataFrame({'src': sources, 'ref': references, 'extra_ref': extra_references, **get_system_outputs('en-de')})
translations_data = pd.DataFrame({'src': sources, 'ref': references, **get_system_outputs('{}'.format(lp))})
translations_data.head()

,src,ref,GTCOM-Primary.6998,JUMT.6616,MSRA.MASS.6945,NEU.6759,online-A.0,online-B.0,online-G.0,online-X.0,TartuNLP-c.6908,tilde-c-nmt.6876,tilde-nc-nmt.6881
0,"""MG Baltic"" byla: naujasis Gustainio advokatas...",MG Baltic case: a new advocate of Mr. Gustaini...,Mg Baltic case: New lawyer for Gustainis says ...,""" MG Baltic "" case : New permanent lawyer says...",MG Baltic case: the new Gustainis lawyer says ...,MG Baltic case: Gustainis's new lawyer says he...,"Case of ""MG Baltic"": New Gustainis lawyer says...","""MG Baltic"" Case: The new Gustainio lawyer say...","""MG Baltic"" file: new Gustainio lawyer says he...","""Mg Baltic"" Case: the new gustainio lawyer say...","""Mg Baltic"" case: new Gustainian lawyer says h...",MG Baltic case: Gustainis' new lawyer says he ...,Mg Baltic case: Gustainis' new lawyer says he ...
1,"Į pirmadienį vykstantį vienuoliktąjį ""MG Balti...",All of the defendants gathered to the eleventh...,The eleventh hearing of the MG Baltic case on ...,Monday 11 th MG took all the defendants partic...,All the accused gathered to the eleventh heari...,The eleventh hearing of the MG Baltic case on ...,"In the eleventh session of the ""MG Baltic"" cas...","On Monday, the 11th MG Baltic case trial was a...","On Monday the ongoing eleventh ""MG Baltic"" pro...","On Monday vykstantį vienuoliktąjį ""Mg Baltic"" ...",The eleventh meeting of the mg Baltic Court on...,The eleventh MG Baltic hearing on Monday broug...,The eleventh hearing of the mg Baltic case on ...
2,Teismo posėdis prasidėjo kaltinamųjų parodymais.,The hearing started with testifying by the def...,The hearing began with testimony from defendants.,The prosecution began with the testimony of th...,The hearing began with testimony from the defe...,The trial began with the testimony of the defe...,The hearing began with the testimony of the ac...,The trial began with the testimony of the accu...,A trial began with the accused to the witness ...,The Court's hearing started the testimonies of...,The court session began with exhibitions of th...,The trial began with testimony from the defend...,The hearing began with testimony from the defe...
3,Praėjusiame teismo posėdyje valstybės paskirta...,"In the previous hearing, Aleksandras Doroševas...","At the last hearing, Alexander Doroshev, the s...","In the last hearing , the state has a new the ...","At the last court hearing, Alexander Doroshev,...","At the last court hearing, the state-appointed...","In a previous court hearing, the new member ap...","At the last hearing, the State appointed the n...",In the last court meeting of the state designa...,"At the last hearing, the state appointed the n...","At the last court session, the state-appointed...","At the last hearing, Alexander Doroshev, the s...","At the previous hearing, Mr Gustainis's new la..."
4,"Tuomet ""MG Baltic"" advokatas Linas Belevičius ...","Then, Linas Belevičius, advocate of MG Baltic,...","Then Linas Belevičius, the lawyer of MG Baltic...","Then MG Baltic's lawyer, Linas Bellevueus, sai...","Linas Belevičius, the lawyer of MG Baltic, the...","At the time, Linas Belevičius, the lawyer of M...","Then, the lawyer of MG Baltic, Linas Belevičiu...",Then MG Baltic lawyer Linas Belevičius stated ...,"Then ""MG Baltic"" lawyer Linas Belevičius state...","""Mg Baltic"" lawyer linas belevičius stated tha...","At the time, mg Baltic lawyer Linas Belevičius...","At the time, Linas Belevičius, an attorney at ...","At the time, mg Baltic's lawyer Linas Beleviči..."


## DA scores:

In [10]:
DA_DATA = 'humaneval-seg-da/ad-seg-scores-{}.csv'

In [11]:
def segment_level_da(path, lp):
    with open(path, "r") as fp:
        fp.readline()
        lines = [l.strip() for l in fp.readlines()]
    data = []
    for line in lines:
        sys, sid, raw_score, z, n = line.split()
        data.append({"system": sys, "SID": sid, "raw_score": raw_score, "z_score": z, "annotators": n, "lp": lp})
    return pd.DataFrame(data)

In [12]:
translations_da_data = segment_level_da(DA_DATA.format('{}'.format(lp)), "{}".format(lp))
translations_da_data.head()

,system,SID,raw_score,z_score,annotators,lp
0,online-A.0,965,80,0.518019638369393,2,lt-en
1,online-A.0,172,6,-2.04724221436286,2,lt-en
2,online-A.0,63,90,0.872792085474223,2,lt-en
3,online-A.0,231,33,-1.06063500655901,2,lt-en
4,online-A.0,895,44,-0.597275785890748,2,lt-en


In [13]:
# import sacrebleu

# def run_bleu(ref, mt):
#         bleu = sacrebleu.corpus_bleu([mt], [[ref],])
#         return bleu.score

In [14]:
def merge_submissions_with_human_scores(submissions, human_scores):
    
    def system_in_submissions(system):
        for col in submissions.columns:
            if system in col:
                return True
            if col in system:
                return True
        return False
    
    lp, data, src, ref, hyp, raw_score, z_score, annotators, system_name, sid = [], [], [], [], [], [], [], [], [], []
    extra_ref = []
    for i, row in tqdm(human_scores.iterrows(), total=len(human_scores), desc=list(human_scores.lp)[0]):
        sub_row = submissions.iloc[int(row['SID'])-1] # segments have ids from 1 to 1997
        
        # Not all systems belong to the newstranslation shared task. Some belong
        # to other shared tasks such as unsupervied MT.
        
        if system_in_submissions(row['system']):
            if row['lp'] == 'zh-en':
                system = '.'.join(row['system'].split('.')[:-1])
            else:
                system = row['system']
            if system in ["UAlacant_-_NM", "RBMT.6722"]:
                continue
                
            hyp.append(sub_row[system])
            lp.append(row['lp'])
            data.append('newstest2019')
            src.append(sub_row['src'])
            ref.append(sub_row['ref'])
            #extra_ref.append(sub_row['extra_ref'])
            raw_score.append(float(row['raw_score']))
            z_score.append(float(row['z_score']))
            annotators.append(int(row['annotators']))
            system_name.append(system)
            sid.append(row['SID'])
            
    
    data = {
        "lp": lp,
        "src": src,
        "mt": hyp,
        "ref": ref,
        #"extra_ref": extra_ref,
        "score": z_score,
        "raw_score": raw_score,
        #"bleu_score": [run_bleu(r, m) for r, m in tqdm(zip(ref, hyp), total=len(ref))],
        "annotators": annotators,
        "system": system_name,
        "sid": sid
    }
    return pd.DataFrame(data)

In [15]:
transl_scores_merged = merge_submissions_with_human_scores(translations_data, translations_da_data)
transl_scores_merged.head()

,lp,src,mt,ref,score,raw_score,annotators,system,sid
0,lt-en,Ministrė pirmininkė Theresa May ne kartą atmet...,"The prime Minister, Theresa May, has repeatedl...",The Prime Minister Theresa May rejected a poss...,0.518020,80.0,2,online-A.0,965
1,lt-en,"Štai, kai L. Jamesas yra aikštėje, ""Lakers"" ši...","Here, when L. James is in the square, the Lake...","When L. James was on the ground this season, L...",-2.047242,6.0,2,online-A.0,172
2,lt-en,Provincijos gubernatoriaus atstovas spaudai Ar...,Provincial governor spokesman Ariff Noori indi...,"Arif Noori, a spokesman for the provincial gov...",0.872792,90.0,2,online-A.0,63
3,lt-en,Amerikietis per 25 minutes įmetė 25 taškus (10...,The American in 25 minutes scored 25 points (1...,The American scored 25 points in 25 minutes (1...,-1.060635,33.0,2,online-A.0,231
4,lt-en,Portalui kauno.diena.lt Kauno miesto muziejaus...,For the portal kauno.diena.lt the director of ...,"Gabrielius Sužiedėlis, the Director of the Kau...",-0.597276,44.0,2,online-A.0,895


In [16]:
original_csv_file = "{}/scores.csv".format(lp)
transl_scores_merged.to_csv(original_csv_file, encoding="utf-8")

### Merge with COMET and BLEU scores

In [17]:
comet_scores_file = "comet_scores/{}-scores.csv".format(lp)
comet_df_csv = pd.read_csv(comet_scores_file)

comet_df = comet_df_csv[['src', 'ref', 'mt', 'comet_score', 'bleu_score', 'system', 'sid', 'lp']]
comet_df.head()

,src,ref,mt,comet_score,bleu_score,system,sid,lp
0,"""MG Baltic"" byla: naujasis Gustainio advokatas...",MG Baltic case: a new advocate of Mr. Gustaini...,MG Baltic case: the new Gustainis lawyer says ...,0.667029,23.546901,MSRA.MASS.6945,1,lt-en
1,"Į pirmadienį vykstantį vienuoliktąjį ""MG Balti...",All of the defendants gathered to the eleventh...,All the accused gathered to the eleventh heari...,0.865774,80.619096,MSRA.MASS.6945,2,lt-en
2,Teismo posėdis prasidėjo kaltinamųjų parodymais.,The hearing started with testifying by the def...,The hearing began with testimony from the defe...,0.841353,23.356899,MSRA.MASS.6945,3,lt-en
3,Praėjusiame teismo posėdyje valstybės paskirta...,"In the previous hearing, Aleksandras Doroševas...","At the last court hearing, Alexander Doroshev,...",0.681179,31.664648,MSRA.MASS.6945,4,lt-en
4,"Tuomet ""MG Baltic"" advokatas Linas Belevičius ...","Then, Linas Belevičius, advocate of MG Baltic,...","Linas Belevičius, the lawyer of MG Baltic, the...",0.693672,37.515212,MSRA.MASS.6945,5,lt-en


In [18]:
human_scores = transl_scores_merged[['src', 'mt', 'score', 'raw_score', 'annotators', 'system']]
human_scores[['annotators']].astype('Int64')
human_scores.head()

,src,mt,score,raw_score,annotators,system
0,Ministrė pirmininkė Theresa May ne kartą atmet...,"The prime Minister, Theresa May, has repeatedl...",0.518020,80.0,2,online-A.0
1,"Štai, kai L. Jamesas yra aikštėje, ""Lakers"" ši...","Here, when L. James is in the square, the Lake...",-2.047242,6.0,2,online-A.0
2,Provincijos gubernatoriaus atstovas spaudai Ar...,Provincial governor spokesman Ariff Noori indi...,0.872792,90.0,2,online-A.0
3,Amerikietis per 25 minutes įmetė 25 taškus (10...,The American in 25 minutes scored 25 points (1...,-1.060635,33.0,2,online-A.0
4,Portalui kauno.diena.lt Kauno miesto muziejaus...,For the portal kauno.diena.lt the director of ...,-0.597276,44.0,2,online-A.0


In [19]:
merged_data = pd.merge(human_scores, comet_df, on=['src', 'mt', 'system'], how='outer')
merged_data.head()

,src,mt,score,raw_score,annotators,system,ref,comet_score,bleu_score,sid,lp
0,Ministrė pirmininkė Theresa May ne kartą atmet...,"The prime Minister, Theresa May, has repeatedl...",0.518020,80.0,2,online-A.0,The Prime Minister Theresa May rejected a poss...,0.846751,9.421197,965,lt-en
1,"Štai, kai L. Jamesas yra aikštėje, ""Lakers"" ši...","Here, when L. James is in the square, the Lake...",-2.047242,6.0,2,online-A.0,"When L. James was on the ground this season, L...",0.585259,16.299447,172,lt-en
2,Provincijos gubernatoriaus atstovas spaudai Ar...,Provincial governor spokesman Ariff Noori indi...,0.872792,90.0,2,online-A.0,"Arif Noori, a spokesman for the provincial gov...",0.843994,50.708113,63,lt-en
3,Amerikietis per 25 minutes įmetė 25 taškus (10...,The American in 25 minutes scored 25 points (1...,-1.060635,33.0,2,online-A.0,The American scored 25 points in 25 minutes (1...,0.638296,11.060955,231,lt-en
4,Portalui kauno.diena.lt Kauno miesto muziejaus...,For the portal kauno.diena.lt the director of ...,-0.597276,44.0,2,online-A.0,"Gabrielius Sužiedėlis, the Director of the Kau...",0.539115,21.941317,895,lt-en


In [20]:
final_csv ="{}.csv".format(lp)
merged_data.to_csv(final_csv, encoding="utf-8")

In [21]:
final_pickle = "{}.pickle".format(lp)
merged_data.to_pickle(final_pickle)

In [22]:
filepath=Path(final_pickle)

data = []
with filepath.open('rb') as f:
    data = pickle.load(f)

new_loaded_transl_scores_merged = pd.DataFrame(data)
new_loaded_transl_scores_merged.head()

,src,mt,score,raw_score,annotators,system,ref,comet_score,bleu_score,sid,lp
0,Ministrė pirmininkė Theresa May ne kartą atmet...,"The prime Minister, Theresa May, has repeatedl...",0.518020,80.0,2,online-A.0,The Prime Minister Theresa May rejected a poss...,0.846751,9.421197,965,lt-en
1,"Štai, kai L. Jamesas yra aikštėje, ""Lakers"" ši...","Here, when L. James is in the square, the Lake...",-2.047242,6.0,2,online-A.0,"When L. James was on the ground this season, L...",0.585259,16.299447,172,lt-en
2,Provincijos gubernatoriaus atstovas spaudai Ar...,Provincial governor spokesman Ariff Noori indi...,0.872792,90.0,2,online-A.0,"Arif Noori, a spokesman for the provincial gov...",0.843994,50.708113,63,lt-en
3,Amerikietis per 25 minutes įmetė 25 taškus (10...,The American in 25 minutes scored 25 points (1...,-1.060635,33.0,2,online-A.0,The American scored 25 points in 25 minutes (1...,0.638296,11.060955,231,lt-en
4,Portalui kauno.diena.lt Kauno miesto muziejaus...,For the portal kauno.diena.lt the director of ...,-0.597276,44.0,2,online-A.0,"Gabrielius Sužiedėlis, the Director of the Kau...",0.539115,21.941317,895,lt-en
